<a href="https://colab.research.google.com/github/liron7722/AI-Generated-Text-Detector/blob/Production/tdIdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load CSV
df = pd.read_csv('data.csv')

# Calculate TF-IDF
vectorizer = TfidfVectorizer(max_features=250, ngram_range=(2,4))
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Save to CSV
tfidf_df.to_csv('tfidf.csv', index=False)